# 컨트랙 간의 관련
## 관련의 구분
 * 컨트랙이 2개 이상의 경우 결합이 필요하다. solidity는 객체지향언이므로 "is-a", "has-a"구현 가능하다
 * 1) Dependency, 2) association, 3) aggregation, 4) composition
 
### Dependency는 제한 시간 동안의 has-a 관계
 * 함수 밖에선 끊어진다.
 
### association은 장기간의 has-a 관계
 * 고객과 주문 등 지속적으로 상호작용 하는 경우
 
### aggregation은 전체와 부분의 관계
 * "has - a"관계, 배열과 같은 구성
 * 학생은 강의를 구성하는 요소지만 그 자체로도 존재할 수 있다.
 
### composition은 전체와 부분이고 전체가 부분을 소유한다.
 * 체가 부분을 해제하면 그 자체로 존재하지 못하는 것
 
```


```

## 상대측 컨트랙 객체의 생성
* 대상 컨트랙의 배포여부에 따라 구분된다.

### 동일한 파일의 컨트랙과 결합
```
contract Customer {
}
contract Order {
    Customer c = new Customer(); //객체를 처음부터 생성
    Order() public {
    }
}
```

```
contract Customer {
}
contract Order {
    Customer c;
    Order() public {
        c = new Customer(); //필요한 시점에 객체를 생성
    }
}
```

### 다른 파일의 컨트랙과 결합
* import <<filename>>로 불러온다.

### 이미 배포된 컨트랙과 결합
* 주소를 통해 전달한다 (new 필요 x)
```
C1 c1 = C1(_addressOfC1);
```

## 함수의 호출
```
상대측 객체를 만들고 나면, 함수를 호출한다. 함수는 객체지향에서 하는 방식으로 점연산자 dot operator를 사용하면 된다.

<instance>.functionMethod()
```
    
* 다른 컨트랙을 호출하는 경우 gas 비용이 infinite라고 계산된다. 그 이유는 다른 컨트랙이 얼마나 gas를 사용하게 될지 모르기 때문이다. gas비용은 전송측에서 차감이 된다는 점에 주의하자.

# ----------------------------------------------------------------------------

# 실습 : new 명령어로 컨트랙 조립 composition

new() 명령어로 컨트랙을 생성하려면 컴파일 시점에 그 소스코드를 가쟈올 수 있어야 한다. 즉, 상대 컨트랙이 동일한 파일에 존재하거나 import문으로 상대 컨트랙이 포함되는 경우가 해당이 된다. 그러면 상대 컨트랙이 컴퍼일되어 바이트코드가 포함되게 된다.

## 1단계 컨트랙 개발
* C1과 C2는 서로 강한 관계를 가지고 있다. 
* C2 생성자에서 C1을 가지고 있다. 또는 set() 함수를 만들어 생성 후에 필요한 시점에 C1과의 연관을 만들고 있다.

In [1]:
%%writefile src/C1C2.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.6.4;

contract C1 {
    uint128 v1;
    function set(uint128 _v1) public {
        v1=_v1;
    }
    function get() public view returns(uint128) {
        return v1;
    }
    function get7() public pure returns(uint128) {
        return 7;
    }
}

contract C2 {
    C1 c1;
    constructor() public {   
        c1=new C1();
    }
    function set(uint128 _v1) public {
        c1.set(_v1);
    }
    function get() public view returns(uint128) {
        return c1.get();
    }
    function get7() public view returns(uint128) {
        return c1.get7();
    }
    function getC1Address() public view returns(address) {
        return address(c1);
    }
}

Overwriting src/C1C2.sol


## 2단계 컴파일

In [2]:
!solc src/C1C2.sol --combined-json abi,bin > src/C1C2.json

## 3단계 배포
* 컴파일 하면 C1, C2 모두 abi, bin이 생성된다. 그 중 C2의 abi, bin을 가져와서 배포를 한다. 
* 여기서는 파일 라이브러리에서 제공하는 fs.readFileSync() 함수를 사용해서 JSON파일을 읽어온다.

In [3]:
%%writefile src/C1C2Deploy.js
var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/C1C2.json");
var _json=JSON.parse(_str)

var _abiArray=JSON.parse(_json.contracts["src/C1C2.sol:C2"].abi);
var _bin="0x"+_json.contracts["src/C1C2.sol:C2"].bin;


async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1000000})
        .on('transactionHash', function(hash){
            console.log(">>> transactionHash"+hash);
        })
        .on('receipt', function(receipt){
            console.log(">>> RECEPIT hash: " + receipt.transactionHash + "\n>>> address:" + receipt.contractAddress);
        })
        .on('error', function(error, receipt) {
            console.log(">>> ERROR "+error);
        });
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting src/C1C2Deploy.js


In [4]:
!node src/C1C2Deploy.js

Deploying the contract from 0x6D7f1aB42e1Ea2E20198398fFAbB10C0ee6fE2AF
>>> transactionHash0x3b303cf17bad2c3d1b1ea775fc70361963c6da99aa48b6556085f5f747b7a9ba
>>> RECEPIT hash: 0x3b303cf17bad2c3d1b1ea775fc70361963c6da99aa48b6556085f5f747b7a9ba
>>> address:0x6E28af3cA12a6B688b6EB36e4D1ED921813cA576
---> The contract deployed to: 0x6E28af3cA12a6B688b6EB36e4D1ED921813cA576


## 4단계 사용

In [6]:
%%writefile src/C1C2Use.js
var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/C1C2.json");
var _json=JSON.parse(_str)

var _abiArray=JSON.parse(_json.contracts["src/C1C2.sol:C2"].abi);
var c2 = new web3.eth.Contract(_abiArray, "0x6E28af3cA12a6B688b6EB36e4D1ED921813cA576");

async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("\nBalance before: " + balanceBefore);
    c2.methods.get7().call().then(function(res) {console.log("C1 get7(): " + res)});
    await c2.methods.set(9).send({from: accounts[0],gas:50000}, function(err,res) {
        console.log("setting 9..."+res);
    });
    c2.methods.get().call().then(function(res) {console.log("C1 get(): " + res)});
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("\nBalance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
}

doIt()

Overwriting src/C1C2Use.js


In [7]:
!node src/C1C2Use.js

Account: 0x6D7f1aB42e1Ea2E20198398fFAbB10C0ee6fE2AF

Balance before: 999990171110000000000
C1 get7(): 7
setting 9...0x998c2758d7b0bd33fc1273fe2582cc7b79ff815e717ae56a316ad3420e81b0ac

Balance after: 999990073396000000000
Balance diff: 97713999970304
C1 get(): 9


# ----------------------------------------------------------------------------

# 실습: 컨트랙의 주소를 사용하여 컨트랙 결합 association
* 앞서 new() 명령어는 소스코드를 포함할 수 있는 경우에 사용하였다. 이번에는 이미 배포된 컨트랙을 결합하여 보자. 그렇다면 C2에 C1의 주소를 넘겨주어야 한다. 즉 C1을 배포하고 그 주소를 알야야 한다.



# C1을 배포하고 주소를 구하기
* 컨트랙 개발(C1)

In [8]:
%%writefile src/C1.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.6.4;

contract C1 {
    uint128 v1;
    function set(uint128 _v1) public {
        v1=_v1;
    }
    function get() public view returns(uint128) {
        return v1;
    }
    function get7() public pure returns(uint128) {
        return 7;
    }
}

Overwriting src/C1.sol


* 컴파일(C1)

In [9]:
!solc src/C1.sol --combined-json abi,bin > src/C1.json

* 배포(C1)

In [10]:
%%writefile src/C1Deploy.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/C1.json");
var _json = JSON.parse(_str)

var _abiArray = JSON.parse(_json.contracts["src/C1.sol:C1"].abi);
var _bin = "0x"+_json.contracts["src/C1.sol:C1"].bin;

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1000000})
        .on('transactionHash', function(hash){
            console.log(">>> transactionHash" + hash);
        })
        .on('receipt', function(receipt){
            console.log(">>> RECEPIT hash: " + receipt.transactionHash + "\n>>> address:" + receipt.contractAddress);
        })
        .on('error', function(error, receipt) {
            console.log(">>> ERROR " + error);
        });
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting src/C1Deploy.js


In [11]:
!node src/C1Deploy.js

Deploying the contract from 0x6D7f1aB42e1Ea2E20198398fFAbB10C0ee6fE2AF
>>> transactionHash0x0dcee5c368e919ddb8098416707e96bb71e1339e192e9a1c36e4daaf7f3abe62
>>> RECEPIT hash: 0x0dcee5c368e919ddb8098416707e96bb71e1339e192e9a1c36e4daaf7f3abe62
>>> address:0xde2f24A82B8A97b41e5d1C5FB642755EeFBaDf55
---> The contract deployed to: 0xde2f24A82B8A97b41e5d1C5FB642755EeFBaDf55


* 사용(C1)

In [14]:
%%writefile src/C1Use.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/C1.json");
var _json = JSON.parse(_str)

var _abiArray = JSON.parse(_json.contracts["src/C1.sol:C1"].abi);

var c1 = new web3.eth.Contract(_abiArray, "0xde2f24A82B8A97b41e5d1C5FB642755EeFBaDf55");
async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    c1.methods.get7().call().then(console.log);
    await c1.methods.set(9).send({from: accounts[0],gas:50000});
    c1.methods.get().call().then(console.log);
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("\nBalance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
}

doIt()

Overwriting src/C1Use.js


In [15]:
!node src/C1Use.js

Account: 0x6D7f1aB42e1Ea2E20198398fFAbB10C0ee6fE2AF
Balance before: 999989695940000000000
7

Balance after: 999989648530000000000
Balance diff: 47409999904768
9


## 1단계 컨트랙 개발(C2)
* C1과 C2를 결합하는 소스코드를 구현해보자. 
* C1은 이미 배포가 되었고, 그 주소를 받을 수 있는 기능이 필요하다. 
* 이 경우 C1의 ABI를 모르면 C2를 컴파일을 할 수 없다. ABI는 함수의 호출방식을 정의하고 있어서 예를 들어 C1의 함수 c1.get7()의 ABI를 모르면, C2의 get7()을 컴파일할 수 없게 된다.

* 앞서 C1.sol, C2.sol을 한 파일 안에 적어주지 않고, import문으로 C1을 포함한다. 
```
import ".\C1.sol"
```

In [16]:
%%writefile src/C2.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.6.4;
import "./C1.sol";

contract C2 {
    C1 c1;
    constructor() public {    //0.6 constructor
        c1=new C1();
    }
    function setC1(address _addressOfC1) public {
        c1 = C1(_addressOfC1);
    }
    function set(uint128 _v1) public {
        c1.set(_v1);
    }
    function get() public view returns(uint128) {
        return c1.get();
    }
    function get7() public view returns(uint128) {
        return c1.get7();
    }
    function getC1Address() public view returns(address) {
        return address(c1);
    }
}

Overwriting src/C2.sol


## 2단계 컴파일(C2)

In [17]:
!solc src/C2.sol --combined-json abi,bin > src/C2.json

## 3단계 배포

In [24]:
%%writefile src/C2Deploy.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/C2.json");
var _json = JSON.parse(_str)

var _abiArray = JSON.parse(_json.contracts["src/C2.sol:C2"].abi);
var _bin = "0x"+_json.contracts["src/C2.sol:C2"].bin;


async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1000000})
        .on('transactionHash', function(hash){
            console.log(">>> transactionHash" + hash);
        })
        .on('receipt', function(receipt){
            console.log(">>> RECEPIT hash: " + receipt.transactionHash + "\n>>> address:" + receipt.contractAddress);
        })
        .on('error', function(error, receipt) {
            console.log(">>> ERROR " + error);
        });
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting src/C2Deploy.js


In [25]:
!node src/C2Deploy.js

Deploying the contract from 0x6D7f1aB42e1Ea2E20198398fFAbB10C0ee6fE2AF
>>> transactionHash0x745c87480adbe6c23f982f1302f76573350ceff680d7a204e284e8675fa8c24f
>>> RECEPIT hash: 0x745c87480adbe6c23f982f1302f76573350ceff680d7a204e284e8675fa8c24f
>>> address:0xbe7A2cE9Ea551E560613A9E4649B8e0FfcBF074c
---> The contract deployed to: 0xbe7A2cE9Ea551E560613A9E4649B8e0FfcBF074c


## 4단계 사용

In [26]:
%%writefile src/C2Use.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/C2.json");
var _json = JSON.parse(_str)

var _abiArray = JSON.parse(_json.contracts["src/C2.sol:C2"].abi);

var c2 = new web3.eth.Contract(_abiArray, "0xbe7A2cE9Ea551E560613A9E4649B8e0FfcBF074c");
async function doIt() {
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    console.log("\n--- new C1 ---");
    await c2.methods.getC1Address().call(function(err, c1addr) {
        if(!err) console.log("c1 address by 'new': "+c1addr);
    });
    c2.methods.get7().call().then(function(res) { console.log("get7(): "+res) });
    console.log("\n--- set the above deployed address of C1 ---");
    await c2.methods.setC1("0xde2f24A82B8A97b41e5d1C5FB642755EeFBaDf55").send({from:accounts[0], gas:50000});
    await c2.methods.getC1Address().call(function(err, c1addr) {
        if(!err) console.log("c1 address by 'setC1()': "+c1addr);
    });
    c2.methods.get7().call().then(console.log);
    await c2.methods.set(222).send({from: accounts[0],gas:50000});
    c2.methods.get().call().then(console.log);
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
}

doIt()

Overwriting src/C2Use.js


* 17: 생성자에서 new 명령어로 생성된 C1의 주소를 출력. 단, C2를 배포하고 첫 회 실행할 때만 유효하고, 2회부터는 이전에 실행된 setC1()의 결과인 이전 C1의 주소가 출력된다.
* 22: 생성자에서 설정한 C1을 제거하고, 위에서 블록체인에 배포한 C1의 주소를 사용하여 교체한다.

In [27]:
!node src/C2Use.js

Account: 0x6D7f1aB42e1Ea2E20198398fFAbB10C0ee6fE2AF
Balance before: 999987661506000000000

--- new C1 ---
c1 address by 'new': 0x8f8d53425c89d2eB9672f789B53791D017189103

--- set the above deployed address of C1 ---
get7(): 7
c1 address by 'setC1()': 0xde2f24A82B8A97b41e5d1C5FB642755EeFBaDf55
7
Balance after: 999987550038000000000
Balance diff: 111468000051200
222


* 현재 c1 주소가 다르다. 하지만 2회를 실행하면 같아진다.

In [28]:
!node src/C2Use.js

Account: 0x6D7f1aB42e1Ea2E20198398fFAbB10C0ee6fE2AF
Balance before: 999987550038000000000

--- new C1 ---
c1 address by 'new': 0xde2f24A82B8A97b41e5d1C5FB642755EeFBaDf55

--- set the above deployed address of C1 ---
get7(): 7
c1 address by 'setC1()': 0xde2f24A82B8A97b41e5d1C5FB642755EeFBaDf55
7
Balance after: 999987444170000000000
Balance diff: 105867999969280
222


* 2회부터는 이전에 실행된 setC1()의 결과인 이전 C1의 주소로 설정되어 출력된다.
* 생성자가 최초에만 호출되기 때문이다.

* 다시 생성자를 호출하여 C1의 주소를 설정하려면, C2를 다시 배포한 후 그 주소로 C2Use.js를 실행하면 된다.

# ----------------------------------------------------------------------------

# 실습: 자동차와 엔진의 조립 - new 명령어 사용
* 동일한 파일에 2개의 컨트랙을 넣어서 개발해 보자.

## 1단계 컨트랙 개발

In [29]:
%%writefile src/Car.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.6.4;

contract Car {
    Engine engineObj;
    string private color;
    event PrintLog(address sender, string msg);

    constructor() public {
        engineObj = new Engine();
    }
    function setColor(string memory _color) public {
        color=_color;
    }
    function getColor() public view returns(string memory) {
        return color;
    }
    function getSpeed() public view returns(uint) {
        return engineObj.getSpeed();
    }
    function speedUpBy10() public {
        engineObj.setSpeedUpBy(10);
    }
    function speedDownBy10() public {
        engineObj.setSpeedDownBy(10);   
    }
    function start() public {
        engineObj.on();
        string memory engineStateStr=engineObj.getEngineState()? "on" : "off";
        emit PrintLog(msg.sender, engineStateStr);
    }
}

contract Engine {
    uint constant private MAXSPEED = 200;
    uint private speed;
    bool private engineState;

    constructor() public {
        speed = 0;
        off();
    }
    function on() public {
        engineState = true;
    }
    function off() public {
        engineState = false;
    }
    function getEngineState() public view returns(bool){
        return engineState;
    }
    function setSpeedUpBy(uint _speed) public {
        if(speed < (MAXSPEED - 10) && engineState == true)
            speed += _speed;
    }
    function setSpeedDownBy(uint _speed) public {
        if((speed - 10) > 0  && engineState == true)
            speed -= _speed;
    }
    function getSpeed() public view returns(uint) {
        return speed;
    }
}

Overwriting src/Car.sol


## 2단계 컴파일

In [30]:
!solc src/Car.sol --combined-json abi,bin > src/Car.json

## 3단계 배포

In [31]:
%%writefile src/carDeploy.js
var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/Car.json");
var _json=JSON.parse(_str)

var _abiArray=JSON.parse(_json.contracts["src/Car.sol:Car"].abi);
var _bin="0x"+_json.contracts["src/Car.sol:Car"].bin;

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1000000})
        .on('transactionHash', function(hash){
            console.log(">>> transactionHash"+hash);
        })
        .on('receipt', function(receipt){
            console.log(">>> RECEPIT hash: " + receipt.transactionHash + "\n>>> address:" + receipt.contractAddress);
        })
        .on('error', function(error, receipt) {
            console.log(">>> ERROR "+error);
        });
    console.log("---> The contract deployed to: " + deployed.options.address)
}
deploy()

Overwriting src/carDeploy.js


In [32]:
!node src/carDeploy.js

Deploying the contract from 0x6D7f1aB42e1Ea2E20198398fFAbB10C0ee6fE2AF
>>> transactionHash0xa3c6fd8325ccfc400441ac7e270dadcfc2f7e69e7dc36a5d6a4c796d988bdc7e
>>> RECEPIT hash: 0xa3c6fd8325ccfc400441ac7e270dadcfc2f7e69e7dc36a5d6a4c796d988bdc7e
>>> address:0xCD66098Abd76b091b9278C9117d8292270DBA294
---> The contract deployed to: 0xCD66098Abd76b091b9278C9117d8292270DBA294


## 4단계 사용 (웹소켓)

In [33]:
%%writefile src/carUse.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.WebsocketProvider("ws://localhost:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/Car.json");
var _json=JSON.parse(_str)

var _abiArray=JSON.parse(_json.contracts["src/Car.sol:Car"].abi);

async function doIt() {
    var car = new web3.eth.Contract(_abiArray, "0xCD66098Abd76b091b9278C9117d8292270DBA294");
    car.events.PrintLog({fromBlock: 'latest', toBlock:'pending'}, function (error, event) {
            console.log(">>> Event fired: " + JSON.stringify(event.returnValues));
        }).on('>> data', function(event){
            console.log(event); // same results as the optional callback above
        }).on('>> changed', function(event){
            console.log(event); // remove event from local database
        }).on('>> error', console.error);

    var speed;
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    car.methods.setColor("RED").send({from: accounts[0], gas:100000});
    car.methods.getColor().call().then(console.log);
    await car.methods.start().send({from: accounts[0], gas:100000})
    await car.methods.speedUpBy10().send({from: accounts[0], gas:100000})
    car.methods.getSpeed().call().then(function(speed) { console.log("-> speed: " + speed) });
    await car.methods.start().send({from: accounts[0], gas:100000})
    await car.methods.speedUpBy10().send({from: accounts[0], gas:100000})
    car.methods.getSpeed().call().then(function(speed) { console.log("-> speed: " + speed) });
    await car.methods.speedDownBy10().send({from: accounts[0], gas:100000})
    car.methods.getSpeed().call().then(function(speed) { console.log("-> speed: " + speed) });
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
    process.exit(1); //force exit -> may terminate some functions (speedDownBy10, getSpeed)
}

doIt()

Overwriting src/carUse.js


* 웹소켓은 계속 이벤트가 발생하는지 지켜보기 때문에 프로세스가 종료하지 않을 수 있다. 마지막에 적은 process.exit(1)로 인해 3번째 speedDownBy10() 호출이 실행하지 않고 강제 종료되고 있다.

In [34]:
!node src/carUse.js

Account: 0x6D7f1aB42e1Ea2E20198398fFAbB10C0ee6fE2AF
Balance before: 999986130200000000000

>>> Event fired: {"0":"0x6D7f1aB42e1Ea2E20198398fFAbB10C0ee6fE2AF","1":"on","sender":"0x6D7f1aB42e1Ea2E20198398fFAbB10C0ee6fE2AF","msg":"on"}
-> speed: 10
>>> Event fired: {"0":"0x6D7f1aB42e1Ea2E20198398fFAbB10C0ee6fE2AF","1":"on","sender":"0x6D7f1aB42e1Ea2E20198398fFAbB10C0ee6fE2AF","msg":"on"}
-> speed: 20
Balance after: 999985636456000000000
Balance diff: 493743999942656


In [35]:
!node src/carUse.js

Account: 0x6D7f1aB42e1Ea2E20198398fFAbB10C0ee6fE2AF
Balance before: 999985636456000000000
RED
>>> Event fired: {"0":"0x6D7f1aB42e1Ea2E20198398fFAbB10C0ee6fE2AF","1":"on","sender":"0x6D7f1aB42e1Ea2E20198398fFAbB10C0ee6fE2AF","msg":"on"}
-> speed: 20
>>> Event fired: {"0":"0x6D7f1aB42e1Ea2E20198398fFAbB10C0ee6fE2AF","1":"on","sender":"0x6D7f1aB42e1Ea2E20198398fFAbB10C0ee6fE2AF","msg":"on"}
-> speed: 30
Balance after: 999985251998000000000
Balance diff: 384458000039936


# ----------------------------------------------------------------------------

# 실습: 사각형과 면적의 조립 - new 명령으로 사각형 생성하고 주소 출력
* 정사각형은 한 변의 길이만 가지면 충분하다. 그리고 각도는 당연하지만 90도를 반환하도록 구현한다. 그리고 별도의 '면적' 컨트랙을 구현하여, 이 정사각형을 조립관계로 가지고, 면적을 계산한다

## 1단계 컨트랙 개발
* getAddressOfSquare() 정사각형의 주소를 출력하는 함수를 구현한다. Square, Area 두 개의 컨트랙이 한 파일에 있고 우리는 Area 컨트랙만 배포한다. 그럼에도 불구하고 Square의 주소를 획득할 수 있다는 점을 유의하자.
* changeSquare(address _addressOfSquare) 별도로 사각형을 배포한 후, 그 사각형의 주소로 변경할 수 있다.

In [36]:
%%writefile src/SquareArea.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.6.4;

contract Square {
    uint128 length;
    function getLength() public view returns(uint128) {
        return length;
    }
    function setLength(uint128 _length) public {
        length=_length;
    }
    function getDegree() public pure returns(uint128) {
        return 90;
    }
}

contract Area {
    Square s;
    address owner;
    constructor() public {
        s = new Square();
        owner = msg.sender;
    }
    function changeSquare(address _addressOfSquare) public {
        s=Square(_addressOfSquare);
    }
    function calcArea() view public returns(uint128) {
        uint128 length = s.getLength();
        uint128 area = length*length;
        return area;
    }
    function setLength(uint128 _length) public {
        s.setLength(_length);
    }
    function getLength() public view returns(uint128) {
        return s.getLength();
    }
    function getDegree() public view returns(uint128) {
        return s.getDegree();
    }
    function getAddressOfSquare() public view returns(address) {
        return address(s);
    }
}

Overwriting src/SquareArea.sol


## 2단계 컴파일

In [37]:
!solc src/SquareArea.sol --combined-json abi,bin > src/SquareArea.json

## 3단계 컨트랙 배포

In [38]:
%%writefile src/SquareAreaDeploy.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/SquareArea.json");
var _json=JSON.parse(_str)

var _abiArray = JSON.parse(_json.contracts["src/SquareArea.sol:Area"].abi);
var _bin = "0x" + _json.contracts["src/SquareArea.sol:Area"].bin;

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1000000})
        .on('transactionHash', function(hash){
            console.log(">>> transactionHash"+hash);
        })
        .on('receipt', function(receipt){
            console.log(">>> RECEPIT hash: " + receipt.transactionHash + "\n>>> address:" + receipt.contractAddress);
        })
        .on('error', function(error, receipt) {
            console.log(">>> ERROR "+error);
        });
    console.log("---> The contract deployed to: " + deployed.options.address)
}

deploy()

Overwriting src/SquareAreaDeploy.js


In [39]:
!node src/SquareAreaDeploy.js

Deploying the contract from 0x6D7f1aB42e1Ea2E20198398fFAbB10C0ee6fE2AF
>>> transactionHash0xcbf248b1bfa5f1b424331100bab26397d63bd9c0e3687bacbab8a77f0714bded
>>> RECEPIT hash: 0xcbf248b1bfa5f1b424331100bab26397d63bd9c0e3687bacbab8a77f0714bded
>>> address:0xEd991B6B8D57e57fec47F41694e6781Fc1c57960
---> The contract deployed to: 0xEd991B6B8D57e57fec47F41694e6781Fc1c57960


## 4단계 사용

In [40]:
%%writefile src/SquareAreaUse.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.WebsocketProvider("ws://localhost:8345"));
var fs=require('fs');
var _str = fs.readFileSync("src/SquareArea.json");
var _json = JSON.parse(_str)

var _abiArray = JSON.parse(_json.contracts["src/SquareArea.sol:Area"].abi);

async function doIt() {
    var area = new web3.eth.Contract(_abiArray, "0xEd991B6B8D57e57fec47F41694e6781Fc1c57960");
    var speed;
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("Balance before: " + balanceBefore);
    area.methods.getDegree().call().then(console.log);
    
    await area.methods.setLength(9).send({from: accounts[0]});
    area.methods.getLength().call().then(console.log);
    area.methods.calcArea().call().then(console.log);
    area.methods.getAddressOfSquare().call().then(function(address) {
        console.log("Square Address: " + address);
    });
    
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
    process.exit(1); //force exit -> may terminate some functions (speedDownBy10, getSpeed)  
}

doIt()

Overwriting src/SquareAreaUse.js


* 웹 소켓으로 실행시
```
process.exit(1)
```. 은 필수

* 처음 실행하면 길이는 0이다. 그 다음 실행하면 9가 된다. 그 이유는 async방식이라 마이닝하지 않고 연달아 시행하면 마이닝 전의 결과가 나오기 때문에 그렇다.

In [41]:
!node src/SquareAreaUse.js

Account: 0x6D7f1aB42e1Ea2E20198398fFAbB10C0ee6fE2AF
Balance before: 999984211772000000000
90
Balance after: 999984114014000000000
Balance diff: 97758000054272


* 즉 process.exit(1) 때문에 
```
Account: 0x0A2aca05EB30707F09C883A4b1881F775ACA4Fa8
Balance before: 99973924640000000000
90
9
81
Square Address: 0x748929714418AaF173Cc14cB269BA246573a71eD
Balance after: 99973043860000000000
Balance diff: 880780000002048
```
* 이렇게 안나온다.

* 아래에서 구한 Square의 주소는 다음 문제에서 changeSquare(address _addressOfSquare)에 설정하여 사용해보자. process.exit(1)으로 일부 명령문이 실행되지 못하고 중단될 수 있다. 명령창에서 실행하고 모든 명령이 실행되면, Ctrl-C를 눌러 강제로 종료하도록 한다.

# ----------------------------------------------------------------------------

# 실습: 사각형과 면적의 조립 - import문으로 사각형 포함, 이미 배포된 사각형 컨트랙으로 교체해보기
* Square.sol을 구현
* Area.sol을 구현
    * import문을 사용하여 Square.sol을 포함
    * changeSquare(address _addressOfSquare) 함수를 호출하여, 앞서 획득한 Square.sol의 주소로 '중도에' 교체해보자.

## 1단계 컨트랙 개발
### Square
```
Square.sol은 별도로 로컬 파일에 저장해서, Area.sol에서 포함하도록 한다. Square.sol은 컴파일하지 않는다.
```.

In [43]:
%%writefile src/Square.sol
//SPDX-License-Identifie
pragma solidity ^0.6.4;

contract Square{
    uint128 length;
    function getLength() public view returns(uint128){
        return length;
    }
    function setLength(uint128 _length) public{
        length = _length;
    }
    function getDegree() public pure returns(uint128){
        return 90;
    }
}

Writing src/Square.sol


### Area 
```
import "./Square.sol"
```.

* Square를 import할 때는 컴파일하는 Area.sol 기준으로 상대 경로

In [48]:
%%writefile src/Area.sol
//SPDX-License-Identifier: GPL-3.0-or-later
pragma solidity ^0.6.4;

import "src/Square.sol";

contract Area{
    Square s;
    address owner;
    
    constructor() public {
        s = new Square();
        owner = msg.sender;
    }
    function changeSquare(address _addressOfSquare) public {
        s = Square(_addressOfSquare);
    }
    function calcArea() view public returns(uint128){
        uint128 length = s.getLength();
        uint128 area = length * length;
        return area;
    }
    function setLength(uint128 _length) public{
        s.setLength(_length);
    }
    function getLength() public view returns(uint128){
        return s.getLength();
    }
    function getDegree() public view returns(uint128){
        return s.getDegree();
    }
    function getAddressOfSquare() public view returns(address){
        return address(s);
    }
}

Overwriting src/Area.sol


## 2단계 컴파일
* import 문으로 포함했으니, Square를 컴파일해서 abi, bin를 필요로 하지 않는다.

In [49]:
!solc src/Area.sol --combined-json abi,bin > src/Area.json

## 3단계 컨트랙 배포
* 컴파일하고 Area.sol의 abi, bin만을 넣어준다. Square.sol의 abi,bin은 무시한다.

In [57]:
%%writefile src/AreaDeploy.js
var Web3 = require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var fs = require('fs');
var _str = fs.readFileSync("src/Area.json");
var _json = JSON.parse(_str)

var _abiArray = JSON.parse(_json.contracts["src/Area.sol:Area"].abi);
var _bin = "0x" + _json.contracts["src/Area.sol:Area"].bin;

async function deploy() {
    const accounts = await web3.eth.getAccounts();
    console.log("Deploying the contract from " + accounts[0]);
    var deployed = await new web3.eth.Contract(_abiArray)
        .deploy({data: _bin})
        .send({from: accounts[0], gas: 1000000}, function(err, transactionHash) {
            if(!err) console.log("hash: " + transactionHash); 
        })
    console.log("---> The contract deployed to: " + deployed.options.address)
}

deploy()

Overwriting src/AreaDeploy.js


In [58]:
!node src/AreaDeploy.js

Deploying the contract from 0x6D7f1aB42e1Ea2E20198398fFAbB10C0ee6fE2AF
hash: 0x1322d519095f80395e4b8904731d7f94a1e213829a425b68549b4cd56a264c9a
---> The contract deployed to: 0x1dEe4aE1a09255d1200b52a7a6c4A10f155d69D9


## 4단계 사용

In [59]:
%%writefile src/AreaUse.js
var Web3=require('web3');
var web3=new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));       //nok
//var web3 = new Web3(new Web3.providers.WebsocketProvider("http://117.16.44.45:8345"));  //ok
//var web3 = new Web3(new Web3.providers.WebsocketProvider("ws://117.16.44.45:8345"));  //ok
var fs=require('fs');
var _str = fs.readFileSync("src/Area.json");
var _json = JSON.parse(_str)

var _abiArray = JSON.parse(_json.contracts["src/Area.sol:Area"].abi);

async function doIt() {
    var hello = new web3.eth.Contract(_abiArray, "0x1dEe4aE1a09255d1200b52a7a6c4A10f155d69D9");
    const accounts = await web3.eth.getAccounts();
    console.log("Account: " + accounts[0]);
    const balanceBefore = await web3.eth.getBalance(accounts[0]);
    console.log("\nBalance before: " + balanceBefore);
    // do by the Square address as set in the constructor
    hello.methods.getAddressOfSquare().call(function(err, c1addr) {
        if(!err) console.log("\n>> Square address by 'new': "+c1addr);
    });
    await hello.methods.setLength(10).send({from: accounts[0]});
    hello.methods.getLength().call().then(console.log);
    hello.methods.calcArea().call().then(console.log);
    hello.methods.getDegree().call().then(console.log);
    hello.methods.getAddressOfSquare().call().then(console.log);
    //redo by the Square address as changed by changeSquare()
    await hello.methods.changeSquare('0xEd991B6B8D57e57fec47F41694e6781Fc1c57960').send({from: accounts[0]});
    hello.methods.getAddressOfSquare().call(function(err, c1addr) {
        if(!err) console.log("\n>> Square address by 'changeSquare: "+c1addr);
    });
    await hello.methods.setLength(10).send({from: accounts[0]});
    hello.methods.getLength().call().then(console.log);
    hello.methods.calcArea().call().then(console.log);
    hello.methods.getDegree().call().then(console.log);
    hello.methods.getAddressOfSquare().call().then(console.log);
    const balanceAfter = await web3.eth.getBalance(accounts[0]);
    console.log("\n Balance after: " + balanceAfter);
    console.log("Balance diff: " + (balanceBefore - balanceAfter));
}

doIt()

Overwriting src/AreaUse.js


* 3: 이벤트를 출력하는 것은 이미 해보았고, 여기서는 사각형 주소 교체가 주목적이므로 그냥 Http를 사용하자.
* 19: 생성자에서 할당된 Square 주소, 즉 import ./Square.sol에서 가져온 new Square()의 주소. 주소 없이 new Square()라고 해도 문제없이 실행이 된다. 즉, import문을 사용하면 주소없이 컨트랙을 생성해서 사용할 수 있다는 의미이다.
* 22: 길이 설정 후에
* 24: 설정된 길이로 면적을 계산
* 28: SquareArea.sol의 배포 주소
 * 첫 번째 getAddressOfSquare() 함수는 new 명령어로 생성된 사각형의 주소를 출력한다.
 * changeSquare() 후 두 번째 getAddressOfSquare()는 web3에서 주입한 사각형의 주소(앞의 예제에서 획득)를 출력한다.

In [60]:
!node src/AreaUse.js

Account: 0x6D7f1aB42e1Ea2E20198398fFAbB10C0ee6fE2AF

Balance before: 999981565092000000000

>> Square address by 'new': 0x46B418C2DDfB86016Ef924BE39995fC58bb6AcD9
0x46B418C2DDfB86016Ef924BE39995fC58bb6AcD9
90
10
100

>> Square address by 'changeSquare: 0xEd991B6B8D57e57fec47F41694e6781Fc1c57960

 Balance after: 999981345186000000000
Balance diff: 219906000027648
0xEd991B6B8D57e57fec47F41694e6781Fc1c57960
90
10
100
